<font  size=5 > HW3 :<br> Use LSTM & CNN model to classify customized candlestick pattern (at least 3 classes) </font>


 r08723063 謝昌哲

所有檔案: <br>candlestick_cnn_r08723063_謝昌哲.py <br> candlestick_lstm_r08723063_謝昌哲.py

<font  size=4 >1. Use LSTM model to classify customized candlestick pattern</font>

candlestick_lstm_r08723063_謝昌哲.py

In [1]:
from sklearn.metrics import confusion_matrix
import pickle
import keras
from keras.layers import LSTM
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes):
    x_train = x_train.reshape(-1, n_step, n_input)
    x_test = x_test.reshape(-1, n_step, n_input)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    return (x_train, x_test, y_train, y_test)

def lstm_model(n_input, n_step, n_hidden, n_classes):
    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None, n_step, n_input), unroll=True))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    return model


def train_lstm(model, x_train, y_train, x_test, y_test, 
        learning_rate, training_iters, batch_size):
    adam = Adam(lr=learning_rate)
    model.summary()
    model.compile(optimizer=adam,
        loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train,
        batch_size=batch_size, epochs=training_iters,
        verbose=1, validation_data=(x_test, y_test))

def print_result(data, x_train, x_test, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(x_train)
    test_pred = model.predict_classes(x_test)
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

def lstm_main():
    # training parameters
    learning_rate = 0.001
    training_iters = 20
    batch_size = 128
    
    #learning_rate = 0.001
    #training_iters = 10
    #batch_size = 128

    # model parameters
    n_input = 40
    n_step = 10
    n_hidden = 512
    n_classes = 10
    
    #n_input = 40
    #n_step = 10
    #n_hidden = 256
    #n_classes = 10
    
    data = load_pkl('C://Users/user/Desktop/HW2/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl')
    x_train, y_train, x_test, y_test = data['train_gaf'], data['train_label'][:, 0], data['test_gaf'], data['test_label'][:, 0]
    x_train, x_test, y_train, y_test = lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes)

    model = lstm_model(n_input, n_step, n_hidden, n_classes)
    train_lstm(model, x_train, y_train, x_test, y_test, learning_rate, 
               training_iters, batch_size)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('LSTM test accuracy:', scores[1])
    print_result(data, x_train, x_test, model)
    
    
    

    
    

In [2]:
lstm_main()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               1132544   
_________________________________________________________________
dense (Dense)                (None, 10)                5130      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 1,137,674
Trainable params: 1,137,674
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
118/118 [==============================] - 22s 187ms/step - loss: 1.9666 - accuracy: 0.2371 - val_loss: 1.4497 - val_accuracy: 0.3452
Epoch 2/20
118/118 [==============================] - 22s 182ms/step - loss: 1.3665 - accuracy: 0.3901 - val_loss: 1.6955 - val_accuracy: 0.2972
Epoch 3/20
118/118 [==============================] - 23s 195ms/step - loss: 1.2211 - 

Accuracy: 0.843

<font  size=4 >2. Use CNN model to classify customized candlestick pattern</font>

candlestick_cnn_r08723063_謝昌哲.py

In [4]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle

from keras import backend as K
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Activation, MaxPool2D


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def get_cnn_model(params):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(10, 10, 4)))
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    return model

def train_model(params, data):
    model = get_cnn_model(params)
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])
    hist = model.fit(x=data['train_gaf'], y=data['train_label_arr'],
                     batch_size=params['batch_size'], epochs=params['epochs'], verbose=2)
    return (model, hist)

def print_result(data, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(data['train_gaf'])
    test_pred = model.predict_classes(data['test_gaf'])
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)
    

In [5]:
PARAMS = {}
PARAMS['pkl_name'] = 'C://Users/user/Desktop/HW2/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl'
PARAMS['classes'] = 10
PARAMS['lr'] = 0.01
PARAMS['epochs'] = 20
PARAMS['batch_size'] = 32
#PARAMS['classes'] = 9
#PARAMS['lr'] = 0.01
#PARAMS['epochs'] = 10
#PARAMS['batch_size'] = 64
PARAMS['optimizer'] = optimizers.SGD(lr=PARAMS['lr'])

# ---------------------------------------------------------
# load data & keras model
data = load_pkl(PARAMS['pkl_name'])
# train cnn model
model, hist = train_model(PARAMS, data)
# train & test result
scores = model.evaluate(data['test_gaf'], data['test_label_arr'], verbose=0)
print('CNN test accuracy:', scores[1])
print_result(data, model)

Epoch 1/20
469/469 - 7s - loss: 1.2363 - accuracy: 0.5338
Epoch 2/20
469/469 - 7s - loss: 0.6067 - accuracy: 0.7773
Epoch 3/20
469/469 - 7s - loss: 0.5042 - accuracy: 0.8145
Epoch 4/20
469/469 - 7s - loss: 0.4612 - accuracy: 0.8302
Epoch 5/20
469/469 - 7s - loss: 0.4313 - accuracy: 0.8411
Epoch 6/20
469/469 - 7s - loss: 0.4088 - accuracy: 0.8507
Epoch 7/20
469/469 - 7s - loss: 0.3888 - accuracy: 0.8588
Epoch 8/20
469/469 - 7s - loss: 0.3769 - accuracy: 0.8643
Epoch 9/20
469/469 - 7s - loss: 0.3602 - accuracy: 0.8687
Epoch 10/20
469/469 - 7s - loss: 0.3498 - accuracy: 0.8720
Epoch 11/20
469/469 - 7s - loss: 0.3391 - accuracy: 0.8768
Epoch 12/20
469/469 - 7s - loss: 0.3270 - accuracy: 0.8856
Epoch 13/20
469/469 - 7s - loss: 0.3155 - accuracy: 0.8879
Epoch 14/20
469/469 - 7s - loss: 0.3073 - accuracy: 0.8896
Epoch 15/20
469/469 - 7s - loss: 0.3013 - accuracy: 0.8902
Epoch 16/20
469/469 - 7s - loss: 0.2919 - accuracy: 0.8960
Epoch 17/20
469/469 - 7s - loss: 0.2858 - accuracy: 0.8972
Epoch 

Accuracy: 0.877